# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Action Points

## 1. No need to make 10 transactions a day. Just one is enough
## 2. See the lema gap size on the losing months
## 3. Can I use actual tick value (close) instead of lema for moves?

In [2]:
%%time
data = run_engine(data)

100%|██████████████████████████████████| 40821/40821 [00:12<00:00, 3208.78it/s]


pl    0.00403
dtype: float64
--------------------------------------
            DateTime_frmt  order_side  close_type       pl
34899 2022-06-22 10:30:00         NaN  sema_close -0.00177
35150 2022-06-23 07:25:00         NaN  sema_close -0.00595
35921 2022-06-27 23:40:00         NaN  sema_close -0.00025
36348 2022-06-29 11:15:00         NaN  sema_close -0.00021
36685 2022-06-30 15:20:00         NaN  sema_close  0.00284
37256 2022-07-04 14:55:00         NaN  sema_close -0.00225
37980 2022-07-07 03:15:00         NaN  sema_close  0.01771
38240 2022-07-08 00:55:00         NaN  sema_close -0.00307
38377 2022-07-08 12:20:00         NaN  sema_close -0.00634
38946 2022-07-12 11:45:00         NaN  sema_close  0.00970
39173 2022-07-13 06:40:00         NaN  sema_close -0.00121
39366 2022-07-13 22:45:00         NaN  sema_close -0.00467
39604 2022-07-14 18:35:00         NaN  sema_close -0.00545
40249 2022-07-19 00:20:00         NaN  sema_close  0.00731
40660 2022-07-20 10:35:00         NaN  sema_clo

100%|███████████████████████████████████████| 231/231 [00:00<00:00, 772.42it/s]

Wall time: 3min 12s


In [3]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')
monthly_avg = np.round(data['df_ohlc']['pl'].sum()/len(set(data['df_ohlc']['month_val'])),4)
print(f'monthly_avg : {monthly_avg}')

month_val
1    0.00587
2   -0.01423
3    0.01014
4    0.00956
5   -0.00548
6   -0.00976
7    0.00793
Name: pl, dtype: float64
---------------------
max : 3 : 0.01014
min : 2 : -0.014229999999999996
---------------------
monthly_avg : 0.0006


In [4]:
data['lema_gap_pl_analysis_month'] = 8

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

,index,positive,negative,count_diff,positive_sum,negative_sum,sum_diff


In [5]:
data['lema_gap_pl_analysis_month'] = 3

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

,index,positive,negative,count_diff,positive_sum,negative_sum,sum_diff
0,0.001,1.0,2.0,-1.0,0.01381,-0.00280,0.01101
1,0.002,1.0,2.0,-1.0,0.00241,-0.00495,-0.00254
2,0.004,0.0,2.0,-2.0,0.00000,-0.00369,-0.00369
7,0.006,0.0,1.0,-1.0,0.00000,-0.00300,-0.00300
9,0.007,1.0,0.0,1.0,0.01521,0.00000,0.01521
8,0.008,2.0,0.0,2.0,0.00607,0.00000,0.00607
3,0.009,0.0,1.0,-1.0,0.00000,-0.00338,-0.00338
10,0.010,1.0,0.0,1.0,0.00189,0.00000,0.00189
5,0.011,1.0,1.0,0.0,0.00490,-0.00218,0.00272
6,0.012,0.0,1.0,-1.0,0.00000,-0.00577,-0.00577


In [6]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [7]:
sys.exit()

SystemExit: 

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 6]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
data['df_small']['lema_gap'].describe()

In [ ]:
abs(all['negative_sum']) / all['positive_sum']

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [8]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
month = 2
data['plot_start']  = f"{data['input_year']}-{month}-1"
data['plot_stop']   = f"{data['input_year']}-{month+1}-1"
# data['plot_stop']   = f"{data['input_year']+1}-1-1"
# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']

plot_graph(data)

In [ ]:
x = 252
data['df_ohlc']['lema_angle'][x-3:x+3]

generate_result_report(data)